In [ ]:
import sys
sys.path.append('/content/drive/MyDrive/project4/notebook')
import f0
import librosa
import numpy as np
import shutil


class CompareF0:

  def __init__(self,data_path,test_path,cut_time):
    self.data_path=data_path
    self.test_path=test_path
    self.cut_time=cut_time
    self.data_time=self.sound_time(data_path)
    self.test_time=self.sound_time(test_path)
    #print('sound_time(test):',self.test_time)
    self.gosa=self.get_f0_evaluate(cut_time,data_path,test_path)
    #print("min:",np.amin(np.array(self.gosa)))
    #print("ave_min:",np.amin(np.array(self.gosa))/self.test_time)
    self.same_song=1
    self.song_boder=500
    if np.amin(np.array(self.gosa))/self.test_time > self.song_boder:
      self.same_song=0
    self.norm_gosa=self.norm(self.gosa)
    self.norm_gosa = np.array(self.norm_gosa)#追加
    self.pred_index=0
    for i in range(len(self.norm_gosa)):
      if self.norm_gosa[i]==1:
        self.pred_index=i
        break

  def sum_error(self,l1,l2):
    e=0
    for i in range(len(l1)):
      e=e+abs(l1[i]-l2[i])
    return e
  
  def sound_time(self,path):
    y,sr = librosa.load(path,sr=44100)
    return len(y) / sr

  def get_f0_evaluate(self,cut_time,data_path,test_path):

    c1=f0.F0(data_path,cut_time)
    c2=f0.F0(test_path,cut_time)

    len1=len(c1.f0_list)
    len2=len(c2.f0_list)

    list1=c1.f0_list
    list2=c2.f0_list

    error_list=[]

    for i in range(len1-len2+1):
      error_list.append(self.sum_error(list1[i:i+len2],list2))
    
    return error_list
  
  def norm(self,list1):

    M=max(list1)
    m=np.amin(np.array(list1))

    list2=[]

    for i in range(len(list1)):
      list2.append((list1[i]-m)/(M-m))

    return list2